# Play versus an agent

Play with pretrained agent yourself!

## Imports

In [ ]:
import sys

if '..' not in sys.path: sys.path.append('..')
from src.dnd.game_utils import *
from src.dnd.actions import ActionInstance
from src.dnd.game_board import GameState
from src.agent.agent import DnDAgent
from src.agent.agent_utils import get_states
from src.dnd.game_configs import *

## Load an agent & game config

In [ ]:
board_size, game_config = get_2v2_0_config()

checkpoint_path = '../rnd/2v2-0/trained-agents/agent-gen30-11.1i-600000'
agent = DnDAgent.load_agent(checkpoint_path, strip=True, epsilon=0)

## Play a game

The following cell creates a new game

In [ ]:
game, colormap = decorate_game(generate_balanced_game(board_size, game_config))

In [ ]:
game_over = False

print_game(game, colormap)
input('Press Enter to start')

while not game_over:
    if game.current_player_id == 1:
        _, _, new_coords, action = get_states(game, agent)
        game_over = take_turn(game, new_coords, action, colormap, True)[0] != GameState.PLAYING
        continue
    
    while True:
        move_input = input("It's your turn. Enter the desired coordinates to move to `y x`:")
        new_coords = tuple([int(x) for x in move_input.split()])
        attack_input = input('Enter the coordinates of unit you want to attack `y x`, or press Enter to skip attack:')
        
        action = None
        if attack_input != '':
            unit = game.current_unit
            target_unit = game.board[tuple([int(x) for x in attack_input.split()])]
            action = ActionInstance(unit.actions[0], source_unit=unit, target_unit=target_unit)

        try:
            game_over = take_turn(game, new_coords, action, colormap, False)[0] != GameState.PLAYING
            break
        
        except Exception as e: print(e)